## Workflow for transcribing audio files with whisper.

#### Create the filename

In [ ]:
%pip install -U openai-whisper
import os
import datetime
import subprocess
import whisper, importlib.metadata

#working_folder = "/Users/reblocke/audio_transcripts"
working_folder = "/Users/blocke/audio_transcripts"
os.makedirs(working_folder, exist_ok=True)

timestamp = datetime.datetime.now().strftime("transcription_%y-%m-%d_at_%H-%M.flac")
audio_output_path = os.path.join(working_folder, timestamp)
text_output_path = os.path.splitext(audio_output_path)[0] + ".txt"

print(audio_output_path)
print(text_output_path)

/Users/blocke/audio_transcripts/transcription_25-08-12_at_09-16.flac
/Users/blocke/audio_transcripts/transcription_25-08-12_at_09-16.txt


#### Command to record the transcription

Note, this can be done with the following terminal command:

Needs ffmpeg installed: "brew install ffmpeg" on mac. 

!ffmpeg -f avfoundation -i ":2" -ac 1 -ar 16000 -b:a 192k output.wav   

In [8]:
# Start ffmpeg and enable writing to its stdin.
process = subprocess.Popen(
    ['ffmpeg', '-f', 'avfoundation', '-i', ':2', '-ac', '1', '-ar', '16000', '-b:a', '192k', audio_output_path],
    stdin=subprocess.PIPE
)

... now recording until the box below is run ...

#### Run the below box to stop the recording

In [9]:
process.communicate(input=b'q')



[q] command received. Exiting.

[out#0/flac @ 0x12beabf40] video:0KiB audio:83803KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.009656%
size=   83811KiB time=00:49:26.24 bitrate= 231.5kbits/s speed=   1x    


(None, None)

#### Command to transcribe the audio just recorded

In [ ]:
print("whisper path:", whisper.__file__)
print("openai-whisper version:", importlib.metadata.version("openai-whisper"))

# If you want the new fast model:
model = whisper.load_model("turbo")              # alias for large-v3-turbo on recent versions

result = model.transcribe(audio_output_path, fp16=False)
print(result["text"])
with open(os.path.splitext(audio_output_path)[0] + ".txt", "w", encoding="utf-8") as f:
    f.write(result["text"])

whisper path: /Users/blocke/anaconda3/envs/llm_py311/lib/python3.11/site-packages/whisper/__init__.py
openai-whisper version: 20250625
 Hey everyone, we'll go ahead and get started. My name is Brian Locke. I'm one of the assistant professors in STICU in the Schmidt Chess Clinic. And I'm going to be giving an update on hypercapnic respiratory failure. I do have a conflict of interest with a local startup that's doing machine learning unrelated to this talk. A few folks I'd like to acknowledge in supporting this work, the American Thoracic Society, the NIH, and then I do have a pulmonary critical care medicine seed grant that I'll talk about some of what we're up to with. And then using the Utah Population Database. And so thanks to the funders of that. Of note, for folks who are using this notebook, LM, which it's one of Google's large language models where you can give it resources and it sort of synthesizes them and goes through it. So I put a bunch of the literature that supports thi

#### Delete the output_path file

In [12]:
os.remove(audio_output_path)